In [66]:
import os
os.environ["NEO4J_URI"] = os.environ.get("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.environ.get("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.environ.get("NEO4J_PASSWORD")

In [ ]:
from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI

# graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph = Neo4jGraph()
graph
print(isinstance(graph, Neo4jGraph))


True


In [68]:
graph.query(
    """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv" AS row
MERGE(m:Movies{id:row.movieId}) 
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toInteger(row.imdbRating)  
FOREACH (director in split(row.director,'|')|
    MERGE (p:Person{name:trim(director)})
    MERGE(p)-[:DIRECTED]->(m))
FOREACH (genere in split(row.genres,'|')|
    MERGE(g:Genere{name:trim(genere)})
    MERGE(g)-[:IN_GENERE]->(m))
FOREACH(actor in split(row.actors,'|')|
    MERGE(a:Genere{name:trim(actor)})
    MERGE(a)-[:ACTED_IN]->(m))
"""
)

[]

In [75]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm=ChatOpenAI(model="gpt-4o"), graph=graph, verbose=True,allow_dangerous_requests=True
)
response = chain.run({"query","tell me Who is the director of the movie Waiting to Exhale from your context"})
response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (director:Person)-[:DIRECTED]->(movie:Movies {title: "Waiting to Exhale"})
RETURN director.name AS Director

Full Context:
[{'Director': 'Forest Whitaker'}]

> Finished chain.


'Forest Whitaker is the director of the movie Waiting to Exhale.'